In [224]:
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.indices.create("es0")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [225]:
es.index(index='es0', body={"content": "this",
                            "name": "that"})

es.index(index='es0', body={"content": "this",
                            "name": "this that"})

es.index(index='es0', body={"content": "that",
                            "name": "this"})

es.index(index='es0', body={"content": "fuck",
                            "name": "that this"})

es.index(index='es0', body={"content": "ca",
                            "name": "this that"})

{'_index': 'es0',
 '_type': '_doc',
 '_id': '0MnoNYYBw-CyhYOb1cBF',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 4,
 '_primary_term': 1}

In [226]:
es.search(index='es0', body={
    "query": {
        "query_string": {
            "fields": ["content", "name"],
            "query": "this AND that",
            # 参数含义参考`multi_match`
            "type": "most_fields",
            # 参数含义参考`multi_match`
            "tie_breaker" : 0.2
        }
    }
})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [227]:
es.search(index='es0', body={
    "query": {
        "query_string": {
            # the content is boosted by 5 using ^5 notation
            "fields": ["content^4",  # 参考`multi_match`
                       "name"],
            "query": "this OR that",
        }
    }
})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
  'max_score': 5.8849893,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': 'zsnoNYYBw-CyhYOb1cA1',
    '_score': 5.8849893,
    '_source': {'content': 'that', 'name': 'this'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': 'zMnoNYYBw-CyhYOb1MDg',
    '_score': 3.8416872,
    '_source': {'content': 'this', 'name': 'that'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': 'zcnoNYYBw-CyhYOb1cAu',
    '_score': 3.7628648,
    '_source': {'content': 'this', 'name': 'this that'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': 'z8noNYYBw-CyhYOb1cA-',
    '_score': 0.52197987,
    '_source': {'content': 'fuck', 'name': 'that this'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '0MnoNYYBw-CyhYOb1cBF',
    '_score': 0.52197987,
    '_source': {'content': 'ca', 'name': 'this that'}}]}}

In [228]:
'''
Field names
You can specify fields to search in the query syntax:

where the status field contains active
    status:active

where the title field contains quick or brown
    title:(quick OR brown)

where the author field contains the exact phrase "john smith"
    author:"John Smith"

where the first name field contains Alice (note how we need to escape the space with a backslash)
    first\ name:Alice

where any of the fields book.title, book.content or book.date contains quick or brown (note how we need to escape the * with a backslash):
    book.\*:(quick OR brown)
'''
es.search(index='es0', body=
{
    "query": {
        "query_string": {
            # 匹配:{'content': 'this', 'name': 'that'}}
            # 匹配:{'content': 'fuck', 'name': 'that this'}
            # 不匹配:{'content': 'that', 'name': 'this'}
            "query": "(content:this OR name:this) AND (name:that)"
        }
    }
})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4, 'relation': 'eq'},
  'max_score': 1.3974485,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': 'zcnoNYYBw-CyhYOb1cAu',
    '_score': 1.3974485,
    '_source': {'content': 'this', 'name': 'this that'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': 'zMnoNYYBw-CyhYOb1MDg',
    '_score': 1.2152811,
    '_source': {'content': 'this', 'name': 'that'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': 'z8noNYYBw-CyhYOb1cA-',
    '_score': 0.52197987,
    '_source': {'content': 'fuck', 'name': 'that this'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '0MnoNYYBw-CyhYOb1cBF',
    '_score': 0.52197987,
    '_source': {'content': 'ca', 'name': 'this that'}}]}}

In [229]:
es.search(index='es0', body=
{
    "query": {
        "query_string": {
            # 匹配:{'content': 'fuck', 'name': 'that this'}
            # 不匹配:{'content': 'fuck', 'name': 'this that'}
            "query": 'name:"that this"'
        }
    }
})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.52197987,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': 'z8noNYYBw-CyhYOb1cA-',
    '_score': 0.52197987,
    '_source': {'content': 'fuck', 'name': 'that this'}}]}}

In [230]:
'''
Use the boost operator ^ to make one term more relevant than another. For instance, if we want to find all documents about foxes, but we are especially interested in quick foxes:

quick^2 fox
The default boost value is 1, but can be any positive floating point number. Boosts between 0 and 1 reduce relevance.

Boosts can also be applied to phrases or to groups:

"john smith"^2   (foo bar)^4
'''
es.search(index='es0', body={
    "query": {
        "query_string": {
            "fields": ["content", "name"],
            "query": "this^4 AND that",
            "type": "most_fields"
        }
    }
})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
  'max_score': 4.8068247,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': 'zcnoNYYBw-CyhYOb1cAu',
    '_score': 4.8068247,
    '_source': {'content': 'this', 'name': 'this that'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': 'zMnoNYYBw-CyhYOb1MDg',
    '_score': 3.8416872,
    '_source': {'content': 'this', 'name': 'that'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': 'zsnoNYYBw-CyhYOb1cA1',
    '_score': 2.745544,
    '_source': {'content': 'that', 'name': 'this'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': 'z8noNYYBw-CyhYOb1cA-',
    '_score': 1.3049496,
    '_source': {'content': 'fuck', 'name': 'that this'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '0MnoNYYBw-CyhYOb1cBF',
    '_score': 1.3049496,
    '_source': {'content': 'ca', 'name': 'this that'}}]}}

In [231]:
es.search(index='es0', body=
{
    "query": {
        "query_string": {
            "query": "(content:this^4 OR name:this) AND (name:that)"
        }
    }
})

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 4, 'relation': 'eq'},
  'max_score': 4.0238547,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': 'zcnoNYYBw-CyhYOb1cAu',
    '_score': 4.0238547,
    '_source': {'content': 'this', 'name': 'this that'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': 'zMnoNYYBw-CyhYOb1MDg',
    '_score': 3.8416872,
    '_source': {'content': 'this', 'name': 'that'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': 'z8noNYYBw-CyhYOb1cA-',
    '_score': 0.52197987,
    '_source': {'content': 'fuck', 'name': 'that this'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '0MnoNYYBw-CyhYOb1cBF',
    '_score': 0.52197987,
    '_source': {'content': 'ca', 'name': 'this that'}}]}}

In [232]:
es.indices.delete('es0')

{'acknowledged': True}